# Fitting diffusion model without analytical solution

#  Model fitting:


***
## Models with likelihood function：
### -MLE:  $ \textit{max}\ L(\theta|Y) $
### -Bayesian estimation:$ P(\theta|Y)\propto L(\theta|Y)P(\theta)$


***
## Models withoout likelihood function:
### Replacing the likelihood function with summary statistics
### Simulate data: X, Observed data: Y
### -Point estimation: $ \textit{min}\ S(X,Y) $
### -Approximate bayesian computation: $ P(\theta|Y)\propto S(X, Y)P(\theta)$




  

***

# Commonly used summary statistics in DDM/SSM:
### 1. Chi-square
<img src="chi.jpg" style="zoom:50%">



### 2. KS-test
<img src="ks.jpg" style="zoom:50%">



### 3. Quantile likelihood
    
    
    
    
    
    
### 4. CAF/WLS
<img src="caf.jpg" style="zoom:50%">


### 5. PDA(Probability density approximation)
<img src="kde_fun.jpg" style="zoom:50%">
$$ \large{\mathscr{L}(\theta|x)\ = \ \Sigma \ f(x|X)}$$

----

In [1]:
import numpy as np
import numba
import pandas as pd
from numba import jit,njit
from numba import prange
from KDEpy import FFTKDE
from scipy.optimize import differential_evolution as de

In [2]:
import hddm

----

# Ornstein–Uhlenbeck process model：
$ dx = (v + \lambda X)dt + \sigma dW$




<img src="ou.png" style="zoom:60%">

# Leaky competing accumulator model:
## Binary choice task:
$ dx_{1} = (I_1 - k X_1 - \beta X_2)dt + \sigma dW$


$ dx_{2} = (I_2 - k X_2 - \beta X_1)dt + \sigma dW$

$ x_i= \textit{max}\ (0,\ x_i)$


<img src="lca.jpg" style="zoom:50%">

# OU Model $\approx$  LCA Model:

$ dx = [(I_1-I_2) + (\beta - k) X]dt + \sigma dW$




---

In [3]:
@jit(fastmath=True)
def ou_single(v,a,z,g,ndt,max_steps):
    n_steps = 0
    x = a * z
    b_u = a
    b_l = 0
    dt = 0.001
    while (x > b_l and x < b_u and n_steps < max_steps):
        dx = (v + g*x) * dt + np.sqrt(dt) * np.random.normal()
        x += dx
        n_steps += 1.0
        t = dt * n_steps
    rt = n_steps * dt
    return rt + ndt if x >= b_l else -rt - ndt

@njit(fastmath=True,parallel=True)
def ou_sim(v,a,z,g,ndt,max_steps,nsim):
    data = np.empty(nsim)
    for n in prange(nsim):
        data[n] = ou_single(v,a,z,g,ndt,max_steps)
    return data

# Steps of PDA:
<img src="pda.png" style="zoom:50%">


---

# Silver bandwidth:
<img src='silver.jpg' style="zoom:50%">


<img src='kde.gif' style="zoom:100%">


In [6]:
def log_like(para):
    global sim_np
    v,a,z,g,ndt = para
    real_data = sim_np.copy()
    
    # classify data into correct and incorrect response
    cor_logrt = real_data[real_data['acc']==1]['log_rt']
    incor_logrt = real_data[real_data['acc']==0]['log_rt']
    N = real_data.shape[0]
    N1 = cor_logrt.shape[0]
    N2 = incor_logrt.shape[0]
    
    # non-decision time must be smaller than minRT 
    if ndt > np.min(np.absolute(real_data['rt'])):
        sum_llf = -np.inf
    else:
        # construct pesudo log-likelihood 
        sim_N = 2000
        sim_data = ou_sim(v,a,z,g,ndt,15000,sim_N)
        df = pd.DataFrame({'rt':np.absolute(sim_data),'acc':(np.sign(sim_data)+1)/2})
        df['log_rt'] = 0.0
        df.loc[df.rt > 0.0, 'log_rt'] = np.log(df.loc[df.rt > 0.0, 'rt'])
        acc_df = list(df.groupby('acc'))
        ## in case there is only one response in the simulated data
        if len(acc_df) == 1:
            x,y = FFTKDE(kernel='epa').fit(acc_df[0][1]['log_rt'].to_numpy()).evaluate(2048)
            if acc_df[0][1]['acc'].sample().to_list()[0] == 0:
                sum_llf = np.sum(np.log(np.interp(incor_logrt,x,y)*(N2/N)))
            elif acc_df[0][1]['acc'].sample().to_list()[0] == 1:
                sum_llf = np.sum(np.log(np.interp(cor_logrt,x,y)*(N1/N)))
        else:
            sim_N1 = acc_df[1][1]['log_rt'].to_numpy().shape[0]
            sim_N2 = acc_df[0][1]['log_rt'].to_numpy().shape[0]
            x_ic,y_ic = FFTKDE(kernel='epa',bw='silverman').fit(acc_df[0][1]['log_rt'].to_numpy()).evaluate(2048)
            x_c,y_c = FFTKDE(kernel='epa',bw='silverman').fit(acc_df[1][1]['log_rt'].to_numpy()).evaluate(2048)
            cor_llf = np.log(np.interp(cor_logrt,x_c,y_c))
            incor_llf = np.log(np.interp(incor_logrt,x_ic,y_ic))
            ## p(rt,choice | theta) = p(rt_correct | theta) * p(rt_incorrect | theta) * p(choice \ theta)
            sum_llf  = np.sum(cor_llf) + np.sum(incor_logrt) + np.log(np.power((sim_N1/sim_N),N1) * np.power((sim_N2/sim_N),N2))
    
    return -sum_llf

In [7]:
## simulate data, trial number = 300
v = 0.5
a = 1.9
z = 0.5
g = -0.2
ndt = 0.4
max_steps = 15000
nsim = 300

sim_data = ou_sim(v,a,z,g,ndt,max_steps,nsim)
sim_df = pd.DataFrame({'rt':np.absolute(sim_data),'log_rt':np.log(np.absolute(sim_data)),'acc':(np.sign(sim_data)+1)/2})
sim_np = sim_df.to_records()

In [8]:
log_like([v,a,z,g,ndt])

280.6078536754261

In [9]:
## fit data
bound = [(0,2),(0,2),(0.3,0.7),(-1,1),(0.1,0.5)]
result =de(func = log_like, bounds = bound)

C:\Users\KLIPS\anaconda3\lib\site-packages\KDEpy\bw_selection.py:282: UserWarning: Silverman's rule failed. Too many idential values. Setting bw = 0.00026037783132902033
  warnings.warn(
C:\Users\KLIPS\anaconda3\lib\site-packages\KDEpy\bw_selection.py:282: UserWarning: Silverman's rule failed. Too many idential values. Setting bw = 0.00024993262693512124
  warnings.warn(


In [36]:
result_df = pd.DataFrame(result['x']).T
result_df.columns = ['v','a','z','g','ndt']
result_df['log-likelihood'] = -result['fun']

In [37]:
result_df

,v,a,z,g,ndt,log-likelihood
0,0.455289,1.70759,0.542364,-0.186065,0.493031,-272.662714


***

# Deeping learning and ABC
<img src='deep.png' style="zoom:50%">
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; Cramer et al., 2020

## 1. Deep learning model approximates likelihood function:
<img src='lan.png' style="zoom:100%">
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; Fengler et al., 2021

## 2. Deep learning model approximates parameter posterior distribution:
<img src='gan.png' style="zoom:100%">

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; Ramesh et al., 2022

In [11]:
# Fitting simulated data with HDDM 0.9.5
sim_df['response'] = sim_df['acc']
sim_df['subj_idx'] = 0
model = 'ornstein'
n_samples = 2000
hddm.model_config.model_config[model]

{'doc': 'Model formulation is described in the documentation under LAN Extension.Meant for use with the extension.',
 'params': ['v', 'a', 'z', 'g', 't'],
 'params_trans': [0, 0, 1, 0, 0],
 'params_std_upper': [1.5, 1.0, None, 1.0, 1.0],
 'param_bounds': [[-2.0, 0.3, 0.2, -1.0, 0.001], [2.0, 2.0, 0.8, 1.0, 2]],
 'boundary': <function hddm.simulators.boundary_functions.constant(t=0)>,
 'params_default': [0.0, 1.0, 0.5, 0.0, 0.001],
 'hddm_include': ['z', 'g'],
 'choices': [-1, 1],
 'slice_widths': {'v': 1.5,
  'v_std': 0.1,
  'a': 1,
  'a_std': 0.1,
  'z': 0.1,
  'z_trans': 0.2,
  't': 0.01,
  't_std': 0.15,
  'g': 0.1,
  'g_trans': 0.2,
  'g_std': 0.1}}

In [12]:
hddm.simulators.model_config[model]['hddm_include']

['z', 'g']

In [13]:
hddmnn_model = hddm.HDDMnn(sim_df,
                           informative = False,
                           include = hddm.simulators.model_config[model]['hddm_include'],
                           p_outlier = 0,
                           w_outlier = 0.1,
                           model = model,
                          is_group_model = False)

{'v': 1.5, 'v_std': 0.1, 'a': 1, 'a_std': 0.1, 'z': 0.1, 'z_trans': 0.2, 't': 0.01, 't_std': 0.15, 'g': 0.1, 'g_trans': 0.2, 'g_std': 0.1}
Supplied model_config specifies params_std_upper for  z as  None.
Changed to 10


In [14]:
hddmnn_model.sample(n_samples, burn = 1000)

 [-----------------100%-----------------] 2000 of 2000 complete in 38.6 sec

In [15]:
hddmnn_model.gen_stats()

,mean,std,2.5q,25q,50q,75q,97.5q,mc err
v,0.315275,0.0827841,0.160047,0.258748,0.315424,0.371004,0.477962,0.003733
a,0.960298,0.095541,0.823803,0.882141,0.948006,1.02417,1.17136,0.00850458
z,0.526762,0.0210816,0.481986,0.512725,0.527455,0.541161,0.565442,0.000857071
g,0.0170634,0.510281,-0.934204,-0.358197,-0.0360376,0.427953,0.914117,0.0448579
t,0.424242,0.0248227,0.366175,0.409047,0.427204,0.44205,0.465009,0.001909


In [16]:
(hddmnn_model.bic - 300*5)/2

-317.7683148094533